This is a short comparison of idiomatic `pandas`, `polars` and `ibis`, all using method chaining.

In [1]:
import sys

sys.version_info

sys.version_info(major=3, minor=12, micro=5, releaselevel='final', serial=0)

In [2]:
import ibis
from ibis import _
ibis.options.interactive = True

In [3]:
ibis.__version__

'9.4.0'

In [ ]:
df = ibis.examples.starwars.fetch()
df

## Pandas

In [ ]:
import re

M = 100

_df = (
    df
    .to_pandas()
    .rename(lambda col: re.sub(r'(?<!^)(?=[A-Z])', '_', col).lower(), axis=1)
    .query("height > 100")
    .query("mass > @M")
    .query('hair_color.str.contains("brown") == True') # some values can be `None` so explicit True required
    .query("sex.notna()")
    .assign(hair_color = lambda c: c.hair_color.str.split(','))
    .explode("hair_color")
    .query("birth_year.notna()")
)

ibis.pandas.connect({'_df': _df}).table('_df')

## Polars

In [ ]:
import re
import polars as pl

M = 100

_df = (
    df
    .to_polars()
    .rename(lambda col: re.sub(r'(?<!^)(?=[A-Z])', '_', col).lower())
    .filter(pl.col("height") > 100)
    .filter(pl.col("mass") > M)
    .filter(pl.col("hair_color").str.contains("brown"))
    .filter(pl.col("sex").is_not_null())
    .with_columns(pl.col("hair_color").str.split(",").alias("hair_color"))
    .explode(pl.col("hair_color"))
    .filter(pl.col("birth_year").is_not_null())
)

ibis.polars.connect({'_df': _df}).table('_df')

## ibis

In [ ]:
M = 100
_df = (
    df
    .rename("snake_case")
    .filter(_.height > 100)
    .filter(_.mass > M)
    .filter(_.hair_color.contains("brown"))
    .filter(_.sex.notnull())
    .mutate(hair_color = _.hair_color.split(","))
    .unnest(_.hair_color)
    .drop_null(_.birth_year)
)

_df